In [ ]:
#hide
from drsa import *

# Deep Recurrent Survival Analysis in PyTorch

> This project features a PyTorch implementation of the [Deep Recurrent Survival Analysis](https://arxiv.org/pdf/1809.02403.pdf) model that is intended for use on uncensored sequential data in which the event is known to occur at the last time step for each observation

## Install

## How to use